In [138]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

Q: if they are elected, they have the metrics, if not elected, no metrics?
should I just check elected validators not all the validators?

challenge: needs rounding for the manually calcualted number, otherwise sometimes it will fail. 

In [139]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [175]:
def getMedianRawStakeSnapshot():
    method = "hmy_getMedianRawStakeSnapshot"
    params = []
    return get_information(method, params)['result']

In [176]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [187]:
def M5_test():
    flag = True
    # get the median stake and the upper and lower level 
    result = getMedianRawStakeSnapshot()
    median = float(result['epos-median-stake'])
    lower = median- 0.15*median
    upper = median + 0.15*median
    print("median stake is " + str(median))
    print("lower bond is " + str(lower))
    print("upper bond is " + str(upper))
    
    validator_infos = result['epos-slot-winners']
    count = 0
    for i in validator_infos:
        addr = i['slot-owner']
        stake = float(i['eposed-stake'])
        count = count + 1
        if stake > upper or stake < lower:
            print("validator: " + addr + " bls public key: " + str(s["key"]['bls-public-key']) +\
                " shard: " + str(s["key"]['shard-id'])) 
            print("effective stake is out of range. The effective stake is " + s["key"]['effective-stake'], "shard", s["key"]['shard-id'])
            print("")
            flag = False

    print("total slots verified:", count)
    if flag:
        return True
    else:
        return False

In [188]:
M5_test()

median stake is 3.33874375e+23
lower bond is 2.8379321875e+23
upper bond is 3.8395553125e+23
total slots verified: 148


True